In [5]:
# import necessary library
import pandas as pd

In [2]:
# fetch the file using a link; unix core
!curl -O https://ftp.ncbi.nih.gov/genomes/refseq/vertebrate_mammalian/Homo_sapiens/latest_assembly_versions/GCF_000001405.39_GRCh38.p13/GCF_000001405.39_GRCh38.p13_genomic.fna.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  920M  100  920M    0     0  71.2M      0  0:00:12  0:00:12 --:--:-- 71.5M


In [4]:
!gunzip /content/GCF_000001405.39_GRCh38.p13_genomic.fna.gz

In [13]:
# install the pyfaidx 
!pip install pyfaidx

  Created wheel for pyfaidx: filename=pyfaidx-0.5.9.5-cp37-none-any.whl size=25142 sha256=a2fd8326d5fea9564fbe65efc5150dc7c6622ba2efab382f6a4b9544f137a993
  Stored in directory: /root/.cache/pip/wheels/d8/31/5f/8053c426a420cc407492252723f20e9a9c7e717909d7e08a9e
Successfully built pyfaidx


In [70]:
# import the pyfaidx
from pyfaidx import Fasta

In [58]:
# read all fasta chromosoms and convert it to upper letters
genes = Fasta('/content/GCF_000001405.39_GRCh38.p13_genomic.fna', sequence_always_upper=True)

In [59]:
# import the key of genes 
Fasta_Keys = genes.keys()

In [60]:
# extract the fasta keys from the human fasta file 
df_Fasta_Keys = pd.DataFrame(Fasta_Keys,columns=['Fasta_Keys'])

In [64]:
# Read the file of cluster 1
DF_class1 = pd.read_csv('/content/ensemble_genes_IDs_updated_df3_class1.csv')
DF_class1.head()

,chrom,chromStart,chromEnd,strand,Gene_ID
0,NC_000001.11,944203,959256,-,ENSG00000188976
1,NC_000001.11,1013497,1014540,+,ENSG00000187608
2,NC_000001.11,1232237,1235041,+,ENSG00000176022
3,NC_000001.11,1253912,1273854,-,ENSG00000160087
4,NC_000001.11,1311597,1324660,-,ENSG00000127054


In [65]:
# chromosome code and a subsequence unclutide from 944203 to 959256 
genes['NC_000001.11'][944203:959256]

>NC_000001.11:944204-959256
CAACACAATGGCCCTGCCTCCCACCGCTTTATTTCTTTCGGTTTCGGATGCAAAACAAAAAATTTTAAAAGAAAATGTGACTTCAAAGGAAAGGAACAAATTTTCAAAGACTTGGGGGAGTGAAGGCAGAGCCTGGTGCAGATGGACGAGGTCTGCAGACGGAGGGCAGAGGTGGTGGAAGGGGCCAGGGGCCTGCAGGCCTCCCCCTGGAACTGGGACTGGTCTCGGTCTGCTGACGTCAGGGTCAGCTCCCCCGCGGAGCTGACTTCAGCAGCCCACAGCTGTGGGGCTTCAGCAGCCACACCAGCCCAGCCCAGCCCAGCTCTCGATACGTTTGGTCTTTCATGCTGAAAAATAAATAATAAAGCCTGTCCCGTGTCTACTGCCTCCCCCAACTGCACAGACGCCAGCCTCTAGGCCTGACTGCCAGGGAGGTGGAAACACTGGCCACCAGCCCGGCAGCCCCTACAGGCCCCCCAGATGGGCTGCCTCAGTCGTCCTCTGAGAGCTGCAGATCCTCCAGCTCGTCCTCCGGCCCCTGGGCCAGCTGCTGCAGCTCCCCAGGGGCCAGCCCCGCCTCTGCGTCTGGGTCTCCATCTGCGGGGAGAGATGGAGGCTACATAAATTTTGCTTTATCAGGAAGAAGCCAGCCTTAGAGGTTACTCATCACTAATTAATCACGGCACTAATTAATTTATCCCTGTTGCTGGCTGCCAGAGAACAGAGCATTTGGCCTGGCCTTCCCAGGGAGGGAAAAGCCTGGCCCAGAGCCCCACGCCCCCCGCCCACGTGGCTCTGCCCTCCCGCCAGATGGGCTCACAGGGCCACACCCTCTCACCCCAAGACCATTCACCCTCCGAGTTGCTGCTGTCCTCCTCGCCCTCCTCCTCGTCCTCTTCATCGTCTTCCACCCCATGCCGAGTGCTCAGGGGCCTCAGTATCCCTGAGGAACAAGAAGCAGAGTCCATATGA

In [67]:
# Chromosome 500 in chrome column
genes[DF_class1.iloc[500]['chrom ']]

FastaRecord("NC_000003.12")

In [68]:
# Fetch the sequence based on the start and the end of genes, then replace the gene ensembl with the chromosome
new_fasta = []
for gen in range(len(df_Fasta_Keys)):
  #starting nuectuotide
  start = DF_class1.iloc[gen]['chromStart']
  #ending nuectuotide
  end = DF_class1.iloc[gen]['chromEnd']
  #apply a function based on the strand
  #inverse and complement
  if DF_class1.iloc[gen]['strand'] == '-':
    seq = -genes[DF_class1.iloc[gen]['chrom ']][start:end]
    new_fasta.append('>%s\n%s' % (DF_class1.iloc[gen]['Gene_ID'], seq))
  # same subsequence
  else:
    seq = genes[DF_class1.iloc[gen]['chrom ']][start:end]
    new_fasta.append('>%s\n%s' % (DF_class1.iloc[gen]['Gene_ID'], seq))

In [69]:
# saving the result of mapped ensembl with the selected portion to a fasta file
with open('genes_DF_Class1.fasta', 'w') as f:
    f.write('\n'.join(new_fasta))